# T1 

### Linear Regress

In [ ]:

# Define the objective function to minimize (MSE)
def objective(params):
    predicted = np.dot(X_test, params)
    mse = np.mean((predicted - y_test) ** 2)
    return mse

predictions = {} 

for pair, features_df in features_dict.items():
    
    # Prepare the data
    X = features_df.values  # Feature values
    y = coint_dict[pair].values[59:]  # All values as target

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a linear regression model as an example
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    # print(f"Mean Squared Error for {pair}: {mse}")
    
     # Initialize the parameters for optimization (e.g., as all ones)
    initial_params = np.ones(X_test.shape[1])

    # Use simulated annealing to optimize the parameters
    result = opt.basinhopping(objective, initial_params, niter=100, stepsize=0.5)

    # Get the optimized parameters
    optimized_params = result.x

    # Re-predict using the optimized parameters
    y_pred_optimized = np.dot(X_test, optimized_params)

    # Save the prediction
    predictions[pair] = y_pred_optimized[0]

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred_optimized)
    r_squared_optimized = r2_score(y_test, y_pred_optimized)
    print(f"Mean Squared Error for {pair}: {mse}")
    # print(f'R-squared (R^2) for {pair}: {r_squared_optimized}')

print(predictions)

In [ ]:
import datetime

# Define the objective function to minimize (MSE)
def objective(params):
    predicted = np.dot(X_test, params)
    mse = np.mean((predicted - y_test) ** 2)
    return mse

predictions_today = {} 
predictions_tomorrow = {} 
change_in_predictions = {}

current_datetime = datetime.datetime.now()
current_date_str = current_datetime.strftime('%Y-%m-%d %H:%M:%S %Z')

def standard_linear_regress():
    
    print('Beginning Normal Standard Linear Regression')
    print('')
    for pair, features_df in features_dict.items():
        # Prepare the data
        X = features_df.values[:-2]  # Exclude last two values for today's and tomorrow's prediction
        y = coint_dict[pair].values[59:-2]  # Similarly, exclude the last two values 

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train a linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict for today and tomorrow using the most recent features
        prediction_today = model.predict([features_df.iloc[-2, :].values])[0]
        prediction_tomorrow = model.predict([features_df.iloc[-1, :].values])[0]

        predictions_today[pair] = prediction_today
        predictions_tomorrow[pair] = prediction_tomorrow
        change_in_predictions[pair] = prediction_today - model.predict([features_df.iloc[-3, :].values])[0]

    print(f"Time is {current_date_str}.")
    for pair in predictions_today.keys():
        current_price = round(compute_spread(pair).iloc[-1], 5)
        print(f"For {pair}:")
        print(f"Today's prediction: {predictions_today[pair]:.5f}. Current price: {current_price}")
        print(f"Tomorrow's prediction: {predictions_tomorrow[pair]:.5f}")
        print(f"Change in prediction compared to yesterday: {change_in_predictions[pair]:.5f}")
        print("-----")

standard_linear_regress()

### ARIMA

In [ ]:
#TESTING
for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]
    
    model = ARIMA(train_set, order=(2, 1, 2))
    model_fit_0 = model.fit()
    
    # Convert pandas series to list for rolling window forecast
    past = train_set.tolist()

    # Empty list for storing predictions
    predictions = []

    # Keeping only the first 50 data in the test dataset.
    # You can run on the whole dataset, but it will take time to run.
    test_set = test_set[:50]

    # Perform rolling window forecast
    for i in range(len(test_set)):
        # Define ARIMA model
        model = ARIMA(past, order=(2, 1, 2))
        # Fit the model
        model_fit = model.fit(start_params=model_fit_0.params)
        # Make forecast
        forecast_results = model_fit.forecast()
        pred = forecast_results[0]
        # Append prediction
        predictions.append(pred)
        # Add test value to train set
        past.append(test_set[i])

NameError: name 'coint_dict' is not defined

In [ ]:
# AR AND MA
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
plt.style.use('seaborn-darkgrid')

for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]

    # Empty list to store aic/bic score
    aic_p = []
    bic_p = []

    # p values
    p = range(1, 6)  # [1,2,3,4,5]

    # AIC/BIC score for different values of p
    for i in p:
        # Define the AR model
        model = ARIMA(train_set, order=(i, 1, 0))  
        # Fit the model
        model_fit = model.fit()
        # Get AIC score
        aic_temp = model_fit.aic  
        # Get BIC score
        bic_temp = model_fit.bic
        # Append AIC score
        aic_p.append(aic_temp) 
        # Append BIC score
        bic_p.append(bic_temp) 

    # Plot of AIC/BIC score for AR term
    plt.figure(figsize=(15, 7))
    plt.plot(range(1, 6), aic_p, color='red')
    plt.plot(range(1, 6), bic_p)
    plt.title('Tuning AR term')
    plt.xlabel('p (AR term)')
    plt.ylabel('AIC/BIC score')
    plt.legend(['AIC score', 'BIC score'])
    plt.show() 
    
for pair, data in coint_dict.items():
    print(f"Running through {pair}")
    split = int(len(data)*0.80)
    train_set, test_set = data[:split], data[split:]

    # Empty list to store AIC/BIC score
    aic_q = []
    bic_q = []

    # q values
    q = range(1, 6)

    # AIC/BIC score for different values of q
    for i in q:
        model = ARIMA(train_set, order=(0, 1, i))
        model_fit = model.fit()
        aic_temp = model_fit.aic
        bic_temp = model_fit.bic
        aic_q.append(aic_temp)
        bic_q.append(bic_temp)

    # Plot of AIC/BIC score for MA term
    plt.figure(figsize=(15, 7))
    plt.plot(range(1, 6), aic_q, color='red')
    plt.plot(range(1, 6), bic_q)
    plt.title('Tuning MA term')
    plt.xlabel('q (MA term)')
    plt.ylabel('AIC/BIC score')
    plt.legend(['AIC score', 'BIC score'])
    plt.show()